PokeAPI pokemon types and prefered attributes on generations 1-2

at first we will set all the libraries and variables used along

In [1]:
import requests, json, pandas as pd, plotly.express as px

In [2]:
#base url for usage in functions
base_url = 'https://pokeapi.co/api/v2/'
#library of pokemon info
pokemon_info = {'Name': [], 'Types': [], 'Attribute': [], 'IV': []}

Then, we will do functions to filter the pokemons we wish to use, and functions to gather the info about types and its IV's

In [3]:
#function to gather pokemon per gen(made to also get other gens)
def catch_pokemon_gen(n):
    url = 'https://pokeapi.co/api/v2/generation/{}'.format(n)
    p = requests.get(url)
    info = p.json()
    poke_list = []

    for pokemon in info['pokemon_species']:
        name = pokemon['name']
        poke_list.append(name)

    return poke_list
gen1 = catch_pokemon_gen(1)
gen2 = catch_pokemon_gen(2)

In [4]:
#function to gather pokemon types
def catch_pokemon_type(name):
    url = base_url + 'pokemon/' + name
    p = requests.get(url)
    info = p.json()
    types = [type['type']['name'] for type in info['types']]
    return types

In [5]:
#function to gather pokemon info on its attributes
def catch_pokemon_iv(name):
    url = base_url + 'pokemon/' + name
    p = requests.get(url)
    info = p.json()
    attributes = ['hp', 'attack', 'defense', 'special-attack', 'special-defense', 'speed']
    highest_attribute = attributes[0]
    highest_attribute_iv = 0

    for i in range(len(attributes)):
        iv_value = info['stats'][i]['base_stat']

        if iv_value > highest_attribute_iv:
            highest_attribute_iv = iv_value
            highest_attribute = attributes[i]

    return highest_attribute, highest_attribute_iv

Now, using a function we shall aggregate all info we gather into the dictionary

In [6]:
def catch_all_info(list):
    for pokemon in list:
        types = catch_pokemon_type(pokemon)
        attribute, iv = catch_pokemon_iv(pokemon)
        pokemon_info['Name'].append(pokemon)
        pokemon_info['Types'].append(types)
        pokemon_info['Attribute'].append(attribute)
        pokemon_info['IV'].append(iv)
    return pokemon_info

In [7]:
gen1_info = catch_all_info(gen1)
gen2_info = catch_all_info(gen2)

Here, we create dataframes for each generation we wish to use for our analysis

In [8]:
gen1df = pd.DataFrame(gen1_info)
gen1df.index += 1
gen2df = pd.DataFrame(gen2_info)
gen2df.index += 1

We then combine all pokemons into a single dataframe for a overall view

In [9]:
total = [gen1df, gen2df]
totaldf = pd.concat(total)
totaldf.reset_index()
totaldf

,Name,Types,Attribute,IV
1,bulbasaur,"[grass, poison]",special-attack,65
2,charmander,[fire],speed,65
3,squirtle,[water],defense,65
4,caterpie,[bug],hp,45
5,weedle,"[bug, poison]",speed,50
...,...,...,...,...
247,blissey,[normal],hp,255
248,pupitar,"[rock, ground]",attack,84
249,tyranitar,"[rock, dark]",attack,134
250,bayleef,[grass],defense,80


Then, by isolating the types and its number, we can see the gamefreak devs prefferences towards some pokemon types

In [29]:
total_types = totaldf['Types'].str[0]
total_types = total_types.value_counts()
types_df = pd.DataFrame(total_types).reset_index()
types_df = types_df.astype({'Types':'str', 'count': 'int'})

In [37]:

type_plot = px.pie(types_df, names='Types', values='count')
type_plot.show()


During the first two generations of pokemon, there was a preference of the devs to opt for water pokemons and normal ones

Doing the same with the pokemon attributes

In [40]:
total_attributes = totaldf['Attribute']
total_attributes = total_attributes.value_counts()
attributes_df = pd.DataFrame(total_attributes).reset_index()

In [39]:
attribute_plot = px.bar(attributes_df, x='Attribute', y='count')
attribute_plot.show()

Looking at the graphs we can see that pokemon in the early generations, used to have more physical attacks than special ones